# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,maceio,-9.6658,-35.7353,25.69,88,20,2.06,BR,1712371283
1,1,edinburgh of the seven seas,-37.0676,-12.3116,18.82,93,87,11.38,SH,1712371346
2,2,lompoc,34.6391,-120.4579,10.33,80,0,5.14,US,1712371346
3,3,stanley,54.8680,-1.6985,10.95,94,75,4.63,GB,1712371328
4,4,bredasdorp,-34.5322,20.0403,14.97,79,100,7.58,ZA,1712371347


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    color="City",
    s="Humidity",
    alpha=0.5,
    title="Humidity Map",
    tiles="OSM",
    scale=1,
    frame_width = 750,
    frame_height = 500)

# Display the map
city_data_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df[(city_data_df["Max Temp"] <= 27) & (city_data_df["Max Temp"] >= 19)\
& (city_data_df["Humidity"] < 50) & (city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] > 0)]

# Drop any rows with null values
ideal_city_data_df.dropna().reset_index()

# Display sample data
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,umm kaddadah,13.6017,26.6876,26.52,14,100,7.66,SD,1712371350
94,94,ciudad melchor muzquiz,27.8833,-101.5167,24.10,38,100,5.71,MX,1712371359
269,269,brak,27.5496,14.2714,19.09,34,6,7.65,LY,1712371382
299,299,sao vicente,-23.9631,-46.3919,26.82,26,2,0.71,BR,1712371263
334,334,dadukou,26.5479,101.7054,26.05,30,6,0.68,CN,1712371391
363,363,tshabong,-26.0500,22.4500,22.41,45,17,3.34,BW,1712371395
416,416,tambacounda,13.7539,-13.7586,26.95,39,9,4.45,SN,1712371402
444,444,adrar,20.5022,-10.0711,26.54,13,3,3.40,MR,1712371406
463,463,kutum,14.2000,24.6667,22.71,18,100,4.37,SD,1712371408
474,474,pul-e khumri,35.9446,68.7151,19.52,27,54,2.49,AF,1712371410


In [9]:
hotel_df = ideal_city_data_df.copy()[["City", "Country", "Lat", "Lng", "Humidity"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
26,umm kaddadah,SD,13.6017,26.6876,14,
94,ciudad melchor muzquiz,MX,27.8833,-101.5167,38,
269,brak,LY,27.5496,14.2714,34,
299,sao vicente,BR,-23.9631,-46.3919,26,
334,dadukou,CN,26.5479,101.7054,30,
363,tshabong,BW,-26.0500,22.4500,45,
416,tambacounda,SN,13.7539,-13.7586,39,
444,adrar,MR,20.5022,-10.0711,13,
463,kutum,SD,14.2000,24.6667,18,
474,pul-e khumri,AF,35.9446,68.7151,27,


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df.copy()[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
26,umm kaddadah,SD,13.6017,26.6876,14,
94,ciudad melchor muzquiz,MX,27.8833,-101.5167,38,
269,brak,LY,27.5496,14.2714,34,
299,sao vicente,BR,-23.9631,-46.3919,26,
334,dadukou,CN,26.5479,101.7054,30,
363,tshabong,BW,-26.0500,22.4500,45,
416,tambacounda,SN,13.7539,-13.7586,39,
444,adrar,MR,20.5022,-10.0711,13,
463,kutum,SD,14.2000,24.6667,18,
474,pul-e khumri,AF,35.9446,68.7151,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "apiKey": geoapify_key,
    "categories": categories
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    long = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
umm kaddadah - nearest hotel: No hotel found
ciudad melchor muzquiz - nearest hotel: No hotel found
brak - nearest hotel: فندق براك السياحي
sao vicente - nearest hotel: Pousada Vitória
dadukou - nearest hotel: 锦江之星酒店
tshabong - nearest hotel: No hotel found
tambacounda - nearest hotel: Le relais de Tamba
adrar - nearest hotel: No hotel found
kutum - nearest hotel: No hotel found
pul-e khumri - nearest hotel: قصر عروسی گلستان
murray bridge - nearest hotel: Bridgeport
as sulayyil - nearest hotel: No hotel found
akjoujt - nearest hotel: Hotel Fatima فندق فاطمة


,City,Country,Lat,Lng,Humidity,Hotel Name
26,umm kaddadah,SD,13.6017,26.6876,14,No hotel found
94,ciudad melchor muzquiz,MX,27.8833,-101.5167,38,No hotel found
269,brak,LY,27.5496,14.2714,34,فندق براك السياحي
299,sao vicente,BR,-23.9631,-46.3919,26,Pousada Vitória
334,dadukou,CN,26.5479,101.7054,30,锦江之星酒店
363,tshabong,BW,-26.0500,22.4500,45,No hotel found
416,tambacounda,SN,13.7539,-13.7586,39,Le relais de Tamba
444,adrar,MR,20.5022,-10.0711,13,No hotel found
463,kutum,SD,14.2000,24.6667,18,No hotel found
474,pul-e khumri,AF,35.9446,68.7151,27,قصر عروسی گلستان


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    color="City",
    hover_cols=["Lng", "Lat", "City","Humidity","Hotel Name", "Country"],
    alpha=0.5,
    title="Hotels Map",
    tiles="OSM",
    scale=1,
    frame_width = 750,
    frame_height = 500)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)